In [1]:
import sqlite3
import pandas as pd

## Schema

![title](https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/master/Northwind_ERD.png)

In [2]:
conn = sqlite3.connect('./northwind-SQLite3/Northwind_small.sqlite')

In [7]:
pd.read_sql(
"""
SELECT CustomerId, 
       UnitPrice, 
       AVG(UnitPrice) OVER (PARTITION BY CustomerId) AS avg_order
FROM [Order]
INNER JOIN OrderDetail ON [Order].Id = OrderDetail.OrderId;""", conn)

,CustomerId,UnitPrice,avg_order
0,ALFKI,45.60,26.73750
1,ALFKI,18.00,26.73750
2,ALFKI,12.00,26.73750
3,ALFKI,43.90,26.73750
4,ALFKI,10.00,26.73750
...,...,...,...
2150,WOLZA,4.50,20.63125
2151,WOLZA,28.50,20.63125
2152,WOLZA,10.00,20.63125
2153,WOLZA,7.75,20.63125


In [10]:
pd.read_sql("""
SELECT DISTINCT CustomerID, 
MAX(DISCOUNT) OVER (PARTITION BY CustomerID) AS max_discount
FROM [ORDER]
JOIN OrderDetail ON [ORDER].Id = OrderDetail.OrderId;
""", conn)

,CustomerId,max_discount
0,ALFKI,0.25
1,ANATR,0.00
2,ANTO,0.15
3,AROUT,0.20
4,BERGS,0.25
...,...,...
84,WARTH,0.20
85,WELLI,0.25
86,WHITC,0.25
87,WILMK,0.00


In [21]:
pd.read_sql("""
SELECT * FROM Product LIMIT 1;
""", conn)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0


In [22]:
pd.read_sql("""
SELECT 
    DISTINCT od.ProductID, p.ProductName,
    MIN(od.UnitPrice) OVER (PARTITION BY od.ProductID) AS min_price
FROM OrderDetail od
JOIN Product p
ON od.ProductID = p.ID;
""", conn)

,ProductId,ProductName,min_price
0,1,Chai,14.4
1,2,Chang,15.2
2,3,Aniseed Syrup,8.0
3,4,Chef Anton's Cajun Seasoning,17.6
4,5,Chef Anton's Gumbo Mix,17.0
...,...,...,...
72,73,Röd Kaviar,12.0
73,74,Longlife Tofu,8.0
74,75,Rhönbräu Klosterbier,6.2
75,76,Lakkalikööri,14.4


In [28]:
pd.read_sql("""
SELECT 
    CustomerID, EmployeeID, (UnitPrice * Quantity) AS order_total,
    AVG(UnitPrice * Quantity) OVER (PARTITION BY CustomerId, EmployeeID) AS avg_order_amount
FROM [Order]
JOIN OrderDetail 
ON [Order].Id = OrderDetail.OrderId;""", conn)

,CustomerId,EmployeeId,order_total,avg_order_amount
0,ALFKI,1,825.0,335.550000
1,ALFKI,1,26.0,335.550000
2,ALFKI,1,400.0,335.550000
3,ALFKI,1,91.2,335.550000
4,ALFKI,3,530.0,480.000000
...,...,...,...,...
2150,WOLZA,6,510.0,269.333333
2151,WOLZA,8,54.0,171.500000
2152,WOLZA,8,199.5,171.500000
2153,WOLZA,8,200.0,171.500000


In [38]:
pd.read_sql("""
SELECT 
    ProductId, UnitPrice,
    RANK() OVER (PARTITION BY CustomerId ORDER BY UnitPrice DESC) AS unit_price_rank
FROM [Order]
JOIN OrderDetail 
ON [Order].Id = OrderDetail.OrderId
ORDER BY CustomerID, unit_price_rank;""", conn)

,CustomerId,UnitPrice,unit_price_rank
0,ALFKI,55.00,1
1,ALFKI,45.60,2
2,ALFKI,45.60,2
3,ALFKI,43.90,4
4,ALFKI,25.00,5
...,...,...,...
2150,WOLZA,10.00,12
2151,WOLZA,10.00,12
2152,WOLZA,7.75,14
2153,WOLZA,7.45,15


In [39]:
pd.read_sql("""
SELECT ProductId, 
       OrderDate, 
       Quantity, 
       LAG(Quantity) OVER (PARTITION BY ProductId ORDER BY OrderDate) AS Lag
FROM [Order] 
INNER JOIN OrderDetail ON [Order].Id = OrderDetail.OrderId ;""", conn)

,ProductId,OrderDate,Quantity,Lag
0,1,2012-08-20,45,NaN
1,1,2012-08-30,18,45.0
2,1,2012-09-30,20,18.0
3,1,2012-11-07,15,20.0
4,1,2012-11-14,12,15.0
...,...,...,...,...
2150,77,2014-04-06,30,21.0
2151,77,2014-04-10,18,30.0
2152,77,2014-04-30,10,18.0
2153,77,2014-05-04,28,10.0
